In [102]:
# Retrieving contact maps from PINDER

from pathlib import Path
from torch.utils.data import DataLoader
from pinder.core import PinderLoader
from pinder.core.index.system import PinderSystem
from pinder.core.loader import structure
from pinder.core import get_pinder_location
from torch_geometric.nn import radius as georadius
from tqdm import tqdm
from pinder.core.loader import filters
from biotite.structure import sasa, apply_residue_wise
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
import os
import random
import pandas as pd
import torch
from sklearn.metrics import matthews_corrcoef, precision_score, recall_score, balanced_accuracy_score
from scipy.stats import pearsonr

PINDER_BASE_DIR="/scicore/home/schwede/durair0000/.local/share/"
SPLITS_DIR="/scicore/home/schwede/pudziu0000/projects/gLM/data/PINDER/eubacteria_5_1024_512_species/"
os.environ["PINDER_BASE_DIR"] = PINDER_BASE_DIR
get_pinder_location()

def get_data(
    system_id,
    interface_threshold=8.0,
):
    system = PinderSystem(system_id)
    if system is None:
        return None
    seq_r, seq_l = system.native_R.sequence, system.native_L.sequence
    try:
        sasa_r = apply_residue_wise(
            system.native_R.atom_array, sasa(system.native_R.atom_array), np.nansum
        )
        sasa_l = apply_residue_wise(
            system.native_L.atom_array, sasa(system.native_L.atom_array), np.nansum
        )
    except Exception as e:
        print(e)
        return None
    r_coords = system.native_R.filter("atom_name", mask=["CA"]).coords
    l_coords = system.native_L.filter("atom_name", mask=["CA"]).coords
    if r_coords.shape[0] == len(seq_r) and l_coords.shape[0] == len(seq_l):
        pos_l, pos_r = georadius(
            torch.tensor(r_coords),
            torch.tensor(l_coords),
            r=interface_threshold,
            max_num_neighbors=10000,
        )
        return {
            "id": system_id,
            "pos_r": pos_r,
            "pos_l": pos_l,
            "seq_r": seq_r,
            "seq_l": seq_l,
            "sasa_r": sasa_r,
            "sasa_l": sasa_l,
        }
    return None

def get_atom_data(split, interface_threshold=8.0):
    # Retrieving positions of the residues that are considered to be interacting with each other
    df = pd.read_csv(f"{SPLITS_DIR}/{split}.txt", sep="\t")
    
    ids = {"R": [], "L": []}
    for i, j in df.iterrows():
        # Neglecting negatives
        if(df["label"][i] == 0): continue
        idR = df["protein1"][i]
        idL = df["protein2"][i]
        ids["R"].append(idR)
        ids["L"].append(idL)

    atom_data = []
    
    for i, id_ in enumerate(ids["R"]):
        try:
            struct_r = structure.Structure(f"{PINDER_BASE_DIR}/pinder/2024-02/pdbs/{id_}-R.pdb", pinder_id=id_)
            struct_l = structure.Structure(f"{PINDER_BASE_DIR}/pinder/2024-02/pdbs/{ids['L'][i]}-L.pdb", pinder_id=ids['L'][i])
        except Exception as e:
            print(e)
            continue
        sasa_r = apply_residue_wise(
            struct_r.atom_array, sasa(struct_r.atom_array), np.nansum
        )
        sasa_l = apply_residue_wise(
            struct_l.atom_array, sasa(struct_l.atom_array), np.nansum
        )
        r_coords = struct_r.filter("atom_name", mask=["CA"]).coords
        l_coords = struct_l.filter("atom_name", mask=["CA"]).coords

        dist_m = torch.cdist(torch.tensor(r_coords), torch.tensor(l_coords))

        pos_l, pos_r = georadius(
            torch.tensor(r_coords),
            torch.tensor(l_coords),
            r=interface_threshold,
            max_num_neighbors=10000,
        )

        pos_dists = []
        for j in range(len(pos_r)):
            pos_dists.append(dist_m[pos_r[j]][pos_l[j]].item())
        
        atom_data_el = {
            "id": f"{id_}-{ids['L'][i]}",
            "pos_r": pos_r,
            "pos_l": pos_l,
            "sasa_r": sasa_r,
            "sasa_l": sasa_l,
            "len_r": len(struct_r.sequence),
            "len_l": len(struct_l.sequence),
            "dist_matrix": dist_m,
            "pos_dists": pos_dists
        }
        
        atom_data.append(atom_data_el)

    return atom_data

def get_contact_map(data_dict, padding=None):
    contact_map = np.zeros((data_dict["len_r"], data_dict["len_l"]))

    for i in range(len(data_dict["pos_r"])):
        contact_map[data_dict["pos_r"][i].item()][data_dict["pos_l"][i].item()] = 1

    if(padding):
        contact_map = contact_map[padding:-padding, padding:-padding]
    return contact_map

def get_cj_map(data_dict, cj_path, n, padding=None):
    id_for_file = data_dict["id"].translate(str.maketrans({'_': '-', '-': '_'}))
    cj = np.load(f"{cj_path}/{id_for_file}_fastCJ.npy")
    # Setting the threshold for the outlier count
    m = np.mean(cj)
    s = np.std(cj)
    threshold = m+n*s

    # Retrieving upper-right quadrant and skipping the tokens <+>
    upper_right_quadrant_cj = cj[1:data_dict["len_r"]+1, data_dict["len_r"]+2:]
    upper_right_quadrant_cm = np.where(upper_right_quadrant_cj > threshold, 1, 0)

    if(padding):
        upper_right_quadrant_cj = upper_right_quadrant_cj[padding:-padding, padding:-padding]
    return upper_right_quadrant_cm, upper_right_quadrant_cj

In [103]:
data = get_atom_data("test")

In [63]:
data[0]["dist_matrix"].shape


torch.Size([244, 244])

In [104]:
# Loading the categorical Jacobian
CJ_PATH="/scicore/home/schwede/pudziu0000/projects/gLM/outputs/categorical_jacobians/glm2_cosine/"

mccs = []
precs = []
recs = []
bal_accs = []
perc_cj_bits_in_cm_arr = []
dist_corr = []

for i in range(len(data)):
    contact_map = get_contact_map(data[i], padding=None)
    cj_contact_map, cj = get_cj_map(data[i], CJ_PATH, n=3, padding=None)
    
    # Compute the correlation between the quadrants
    mcc = matthews_corrcoef(contact_map.flatten(), cj_contact_map.flatten())
    prec = precision_score(contact_map.flatten(), cj_contact_map.flatten())
    rec = recall_score(contact_map.flatten(), cj_contact_map.flatten())
    bal_acc = balanced_accuracy_score(contact_map.flatten(), cj_contact_map.flatten())
    mccs.append(mcc)
    precs.append(prec)
    recs.append(rec)
    bal_accs.append(bal_acc)

    # Computing the percentage of cj_cm signals in true_cm
    cj_bits_in_cm = np.where(cj_contact_map == 1, contact_map, 0)
    if(np.sum(cj_contact_map)):
        perc_cj_bits_in_cm = cj_bits_in_cm.sum()/np.sum(cj_contact_map)
    else:
        perc_cj_bits_in_cm = 0
    perc_cj_bits_in_cm_arr.append(perc_cj_bits_in_cm)

    # Computing correlation between distances and cj signals
    dist_corr.append(np.corrcoef(np.array(data[i]["dist_matrix"].flatten()), cj.flatten())[0][1])

/scratch/pudziu0000/slurm-job.41550003/ipykernel_3937195/2735230891.py:34: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  dist_corr.append(np.corrcoef(np.array(data[i]["dist_matrix"].flatten()), cj.flatten())[0][1])
/scratch/pudziu0000/slurm-job.41550003/ipykernel_3937195/2735230891.py:34: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  dist_corr.append(np.corrcoef(np.array(data[i]["dist_matrix"].flatten()), cj.flatten())[0][1])
/scratch/pudziu0000/slurm-job.41550003/ipykernel

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 9900 and the array at index 1 has size 10000

In [105]:
print("MCC: ", np.average(mccs))

# Ability not to label a negative sample as a positive
print("Precision: ", np.average(precs))

# Ability to find all positive samples 
print("Recall: ", np.average(recs))

# Being right overall
print("Balanced accuracy: ", np.average(bal_accs))

# How many matches of outliers in the contact map signal
print("Average percentage of outliers within contact map: ", np.average(perc_cj_bits_in_cm_arr))

# Correlation coefficient between real distances and CJ signals
print("Correlation between distances and CJ: ", np.average(dist_corr))


MCC:  0.017462850955104574
Precision:  0.01891936642708193
Recall:  0.02407872520057646
Balanced accuracy:  0.5109340480143657
Average percentage of outliers within contact map:  0.01891936642708193
Correlation between distances and CJ:  0.0012931095829949292


In [108]:
dist_corr = np.array(dist_corr)

In [115]:
print("Average corr. coeff. for negative corr. coeff: ", np.average(dist_corr[np.where(dist_corr < 0)]))
print("Average corr. coeff. for positive corr. coeff: ", np.average(dist_corr[np.where(dist_corr >= 0)]))

Average corr. coeff. for negative corr. coeff:  -0.017629402882166103
Average corr. coeff. for positive corr. coeff:  0.015131961982888818
